In [4]:
import pandas as pd
import numpy as np

from datetime import datetime

%matplotlib inline
pd.set_option('display.max_rows', 500)


In [5]:
# pd_JH_data=pd.read_csv('../data/processed/COVID_relational_confirmed.csv',sep=';',parse_dates=[0])
# pd_JH_data=pd_JH_data.sort_values('date',ascending=True).reset_index(drop=True).copy()
# pd_JH_data.head()

In [6]:

pd_JH_data_country=pd.read_csv('../data/processed/COVID_relational_database_by_country_confirmed.csv',sep=';',parse_dates=[0])
pd_JH_data_country=pd_JH_data_country.sort_values('date',ascending=True).reset_index(drop=True).copy()
pd_JH_data_country.head()

,date,country,confirmed
0,2020-01-22,Afghanistan,0
1,2020-01-22,Namibia,0
2,2020-01-22,Nepal,0
3,2020-01-22,Netherlands,0
4,2020-01-22,New Zealand,0


In [7]:
# len(set(pd_JH_data["country"]))== len(set(pd_JH_data_country["country"]))

In [8]:
test_data=pd_JH_data_country[((pd_JH_data_country['country']=='Nepal')|
                      (pd_JH_data_country['country']=='Germany'))&
                     (pd_JH_data_country['date']>'2020-03-20')
                   ]


In [9]:
test_data.head()


,date,country,confirmed
11095,2020-03-21,Nepal,1
11273,2020-03-21,Germany,22213
11281,2020-03-22,Nepal,1
11463,2020-03-22,Germany,24873
11472,2020-03-23,Nepal,2


In [10]:
test_data.groupby(['country']).agg(np.max)

,date,confirmed
country,,
Germany,2020-09-11,259735
Nepal,2020-09-11,51919


In [20]:
import numpy as np
from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept=True)
from scipy import signal

def get_doubling_time_via_regression(in_array):
    ''' Use a linear regression to approximate the doubling rate'''

    y = np.array(in_array)
    X = np.arange(-1,2).reshape(-1, 1)

    assert len(in_array)==3
    reg.fit(X,y)
    intercept=reg.intercept_
    slope=reg.coef_

    return intercept/slope




In [21]:
def rolling_reg(df_input,col='confirmed'):
    ''' input has to be a data frame'''
    ''' return is single series (mandatory for group by apply)'''
    days_back=3
    result=df_input[col].rolling(
                window=days_back,
                min_periods=days_back).apply(get_doubling_time_via_regression,raw=False)
    return result
    

In [23]:
test_data[['country','confirmed']].groupby(['country']).apply(rolling_reg,'confirmed')

country       
Germany  11273            NaN
         11463            NaN
         11654       7.417994
         11840       7.142035
         12024       8.012983
         12215       6.954407
         12405       6.501919
         12592       7.390371
         12776      10.136671
         12965      13.541893
         13157      13.781393
         13340      13.140681
         13532      12.037271
         13717      12.735506
         13910      16.052694
         14093      21.372453
         14283      27.427355
         14472      27.511936
         14656      21.792179
         14844      21.495848
         15038      26.565108
         15221      36.198404
         15409      43.982990
         15600      49.423444
         15784      74.043747
         15976      56.424411
         16160      42.468318
         16352      41.525988
         16541      49.898063
         16724      75.684007
         16913      78.000000
         17105      94.526338
         17288      82.98

## applying to large dataset

In [14]:
pd_DR_result_country=pd_JH_data_country[['country','confirmed']].groupby(['country']).apply(rolling_reg,'confirmed').reset_index()

In [12]:
pd_DR_result_country.head()

,country,level_1,confirmed
0,Afghanistan,0,NaN
1,Afghanistan,329,NaN
2,Afghanistan,496,NaN
3,Afghanistan,683,NaN
4,Afghanistan,871,NaN


In [13]:
pd_DR_result_country=pd_DR_result_country.rename(columns={'confirmed':'confirmed_DR',
                             'level_1':'index'})
pd_DR_result_country.tail()

,country,index,confirmed_DR
43611,Zimbabwe,42741,85.333333
43612,Zimbabwe,42909,inf
43613,Zimbabwe,43117,30.328272
43614,Zimbabwe,43264,26.041137
43615,Zimbabwe,43615,112.544529


In [18]:
pd_JH_data_country.head()

,index,date,country,confirmed
0,0,2020-01-22,Afghanistan,0
1,1,2020-01-22,Namibia,0
2,2,2020-01-22,Nepal,0
3,3,2020-01-22,Netherlands,0
4,4,2020-01-22,New Zealand,0


In [15]:
pd_JH_data_country=pd_JH_data_country.reset_index()
pd_JH_data_country.head()

,index,date,country,confirmed
0,0,2020-01-22,Afghanistan,0
1,1,2020-01-22,Namibia,0
2,2,2020-01-22,Nepal,0
3,3,2020-01-22,Netherlands,0
4,4,2020-01-22,New Zealand,0


In [19]:
pd_result_large_country=pd.merge(pd_JH_data_country,pd_DR_result_country[['index','confirmed_DR']],on=['index'],how='left')
pd_result_large_country.tail()

,index,date,country,confirmed,confirmed_DR
43611,43611,2020-09-09,Gambia,3293,67.812500
43612,43612,2020-09-09,Georgia,1773,38.846442
43613,43613,2020-09-09,Germany,256433,181.692436
43614,43614,2020-09-09,Estonia,2585,96.616352
43615,43615,2020-09-09,Zimbabwe,7429,112.544529


## filter data using groupby and apply


In [20]:
from scipy import signal

def savgol_filter(df_input,column='confirmed',window=5):
    ''' Savgol Filter which can be used in groupby apply function 
        it ensures that the data structure is kept'''
    window=5, 
    degree=1
    df_result=df_input
    
    filter_in=df_input[column].fillna(0) # attention with the neutral element here
    
    result=signal.savgol_filter(np.array(filter_in),
                           5, # window size used for filtering
                           1)
    df_result[column+'_filtered']=result
    return df_result

In [21]:
pd_filtered_result_country=pd_JH_data_country[['country','confirmed']].groupby(['country']).apply(savgol_filter).reset_index()

In [22]:
pd_result_large_country=pd.merge(pd_result_large_country,pd_filtered_result_country[['index','confirmed_filtered']],on=['index'],how='left')
pd_result_large_country.tail()

,index,date,country,confirmed,confirmed_DR,confirmed_filtered
43611,43611,2020-09-09,Gambia,3293,67.812500,3295.2
43612,43612,2020-09-09,Georgia,1773,38.846442,1768.0
43613,43613,2020-09-09,Germany,256433,181.692436,256356.2
43614,43614,2020-09-09,Estonia,2585,96.616352,2584.8
43615,43615,2020-09-09,Zimbabwe,7429,112.544529,7504.8


### Filtered doubling rate

In [26]:

pd_filtered_doubling_country=pd_result_large_country[['country','confirmed_filtered']].groupby(['country']).apply(rolling_reg,'confirmed_filtered').reset_index()

pd_filtered_doubling_country=pd_filtered_doubling_country.rename(columns={'confirmed_filtered':'confirmed_filtered_DR',
                             'level_1':'index'})

pd_filtered_doubling_country.tail()

,country,index,confirmed_filtered_DR
43611,Zimbabwe,42741,72.397299
43612,Zimbabwe,42909,50.370316
43613,Zimbabwe,43117,54.005632
43614,Zimbabwe,43264,49.168665
43615,Zimbabwe,43615,42.255331


In [27]:
pd_result_large_country=pd.merge(pd_result_large_country,pd_filtered_doubling_country[['index','confirmed_filtered_DR']],on=['index'],how='left')
pd_result_large_country.tail()

,index,date,country,confirmed,confirmed_DR,confirmed_filtered,confirmed_filtered_DR
43611,43611,2020-09-09,Gambia,3293,67.812500,3295.2,89.527473
43612,43612,2020-09-09,Georgia,1773,38.846442,1768.0,45.161880
43613,43613,2020-09-09,Germany,256433,181.692436,256356.2,182.386651
43614,43614,2020-09-09,Estonia,2585,96.616352,2584.8,108.525424
43615,43615,2020-09-09,Zimbabwe,7429,112.544529,7504.8,42.255331


In [28]:
mask=pd_result_large_country['confirmed']>100
pd_result_large_country['confirmed_filtered_DR']=pd_result_large_country['confirmed_filtered_DR'].where(mask, other=np.NaN) 


In [29]:
pd_result_large_country[pd_result_large_country['country']=='Germany'].tail()

,index,date,country,confirmed,confirmed_DR,confirmed_filtered,confirmed_filtered_DR
42860,42860,2020-09-05,Germany,251058,225.482717,251107.0,209.504918
43044,43044,2020-09-06,Germany,251728,347.436678,252330.4,203.616986
43236,43236,2020-09-07,Germany,253626,196.368640,253560.4,205.700334
43420,43420,2020-09-08,Germany,254957,156.975534,254958.3,193.018278
43613,43613,2020-09-09,Germany,256433,181.692436,256356.2,182.386651


In [30]:
pd_result_large_country.to_csv('../data/processed/COVID_final_set_country.csv',sep=';',index=False)